<h2>Imports</h2>

In [7]:
# Import libraries

import simpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from util import Battery, read_battery_process, solar_charge_process, compute_tasks_process

ModuleNotFoundError: No module named 'util'

<h2>Constraints</h2>

In [ ]:
# Set constraints

max_capacity =
max_solar =

minimal_reliability = 0.99

<h2>Data prep</h2>

In [6]:
# Load and prepare data from csv-files
START_DATE = pd.to_datetime("2021-06-01 00:00:00")
END_DATE = pd.to_datetime("2021-06-08 00:00:00")
INDEX = pd.date_range(START_DATE, END_DATE, freq="s")  # +timedelta(minutes=1)


# Load data between START_DATE and END_DATE
with open("data/weather_berlin_2021-06.csv", "r") as csvfile:
    weather_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]
with open("data/low_prio_tasks.csv", "r") as csvfile:
    tasks_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]
with open("data/high_prio_tasks.csv", "r") as csvfile:
    hptasks_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]

FileNotFoundError: [Errno 2] No such file or directory: 'data/weather_berlin_2021-06.csv'

<h2>Scenario and costs</h2>

In [ ]:
# Set researched values for costs etc. depending on the scenario

cost_per_capacity = 100
cost_per_solar = 100

BATTERY_CAPACITY = 1000     # Wh
BATTERY_INITIAL_SOC = 500   # Wh
SOLAR_AREA = np.linspace(0.1, 5, 50)              # m^2

In [1]:
SOLAR_AREA = np.linspace(0.1, 5, 50)
SOLAR_AREA

NameError: name 'np' is not defined

<h2>Simulation Function</h2>

In [ ]:
# Run simulation

def simulate_timeseries(demand_loads:list, pv_load:list, configuration:list):




    return timeseries_df

<h2>Main</h2>

In [ ]:
for count_p, panel in enumerate(solar_panels):
    for count_b, capacity in enumerate(battery_capacities):

        results[count_p, count_b] = simulation(panel, capacity)

<h2>Results</h2>